# Indexes in Postgres

### Introduction

In this lesson, we'll learn about indexes in Postgres, and the costs and benefits of adding an index.  Let's take a look.

### Reading

Read through section 1.1 of the postgres documentation, [Postgres documentation](https://www.postgresql.org/docs/9.1/indexes-intro.html).  Then we'll review some of what's in that material.

### Takeaways

> Ok, so let's make sure that we understand some of the key points from the documentation.  

Adding an index is a way to speed up our query time.  If we have a table that looks like the following:

| id | title           | genre     |
|----|-----------------|-----------|
| 1  | Maltese Falcoln      |Mystery |
| 2  | Tom Sawyer | Adventure   |
| 3  | Gone Girl       | Mystery   |

And we want to identify all of the mystery books, then we write the following SQL query:

```SQL 
SELECT * FROM books WHERE genre = 'Mystery'
```

And SQL will need to move through each genre to complete the query.  When we index a specific column, we can think of storing the data almost like a dictionary (although the actual underlying data structure will vary):

In [1]:
genre_index = {'Adventure': [1], 'Mystery': [2, 3]}

So now if we wish to find all of the Mystery records, we can do so just by returning the list from our index.

In [2]:
genre_index['Mystery']

[2, 3]

No more going one by one through all of our records.  Instead Postgres will first retrieve the ids of the mystery novels, and then just search for those Ids.

### What's the downside

There are a couple of downsides to indexing.

1. Space

With an index, we now have to store extra information in our database.

2. Transaction Time 

Perhaps even more noticable is the transaction time.  Each time we insert or delete a record, we also need to update the index in our database.  This adds overhead to each database transaction, and thus can slow down what quick operations like creating a user, or creating a new twitter post.  For this reason, we should only use indexes when they are needed.  

### Adding an Index in Postgres

Now let's review how we can create an index in postgres.  Before we dive, let's see if we can do a bit of a before and after.

Let's say we want to add an index to the genre of a movie.  Before we do so, we want to see the time it takes to search for movies by their genre.

```SQL
EXPLAIN ANALYZE SELECT * FROM movies WHERE genre = 'Action';
```

<img src="./explain-analyze.png" width="70%">

Ok, now let's add our index on the movie genres.

```SQL
CREATE INDEX movies_genre_index ON movies (genre);
```

And let's quit out of postgres (`\q`), and log back into the postgres shell.  Then we can re-execute the query.

<img src="./post-index.png" width="90%">

So notice now that instead of a Seq Scan, that we saw before, it is now using a Bitmap Heap Scan -- which is it's mechanism for indexing.  And notice that the execution time decreased from 1.661 ms, to .864 ms.

### Summary

In this lesson, we were introduced to indexing in databases.  We saw that indexing allows us to perform faster queries by allowing us to avoid a sequential scan of the data.  It does so by keeping track of which records have certain values in a column
```python
genre_index = {'Adventure': [1], 'Mystery': [2, 3]}
```

The downside to this is mainly the additional transaction time it takes update the table, as now inserting or deleting records from the database requires maintenance of the index.  To assess the benefit from performing an index, we can use the EXPLAIN ANALYZE command on a query like so:

```
EXPLAIN ANALYZE SELECT * FROM movies WHERE genre = 'Action';
```
And then can see the benefit from adding an index on a column:

```
CREATE INDEX movies_genre_index ON movies (genre);
```

### Resources



[Indexes postgres documentation](https://www.postgresql.org/docs/9.1/indexes.html)

[Indexes and Btree](https://www.youtube.com/watch?v=NI9wYuVIYcA&ab_channel=Ivantalksaboutcomputers)

[Database Indexing - Clustered](https://www.freecodecamp.org/news/database-indexing-at-a-glance-bb50809d48bd/)

[Indexing Wikipedia](https://en.wikipedia.org/wiki/Database_index#Index_architecture_and_indexing_methods)

[Online Book about Indexing](https://use-the-index-luke.com/sql/table-of-contents)